In [ ]:
# cuda监控: 
!watch -n 1 "nvidia-smi --query-gpu=memory.used,memory.total,power.draw,power.limit --format=csv,noheader,nounits | awk -F, '{printf \"RAM: %s/%s MiB | Power: %s/%s W\n\", \$1, \$2, \$3, \$4}'"

在上传bilibili时,可能遇到审查需要裁剪的区间

In [8]:
import subprocess
import json
import bisect
import os
import datetime
import time 

def time_to_seconds(time_str):
    """将 HH:MM:SS 格式的时间字符串转换为总秒数"""
    parts = list(map(float, time_str.split(':')))
    if len(parts) == 3:
        return parts[0] * 3600 + parts[1] * 60 + parts[2]
    elif len(parts) == 2:
        return parts[0] * 60 + parts[1]
    return parts[0]

def seconds_to_time(seconds):
    """将秒数转换为 HH:MM:SS.sss 格式"""
    return str(datetime.timedelta(seconds=seconds))

def get_frame_time(frame):
    """按优先级从帧数据中提取时间戳，以解决不同文件格式导致字段缺失的问题"""
    # 尝试不同字段: pkt_pts_time (常用), pts_time (frame pts), best_effort_timestamp_time (通用)
    for field in ['pkt_pts_time', 'pts_time', 'best_effort_timestamp_time']:
        time_val = frame.get(field)
        if time_val is not None:
            # 找到第一个非None值即返回
            return float(time_val)
    return None


def cut_segment_lossless(input_file: str, t_start_str: str, t_end_str: str, output_file: str):
    """
    无损裁剪 MP4 文件中指定时间段的音视频内容 (基于关键帧)。
    """
    print(f"--- 任务开始: 裁剪区间 {t_start_str} 至 {t_end_str} ---")
    
    # --- 1. 定义和转换时间 ---
    try:
        T_start_sec = time_to_seconds(t_start_str)
        T_end_sec = time_to_seconds(t_end_str)
    except Exception as e:
        print(f"❌ 错误：时间格式不正确。请使用 HH:MM:SS 或 MM:SS 格式。{e}")
        return

    # 生成唯一的临时文件名，防止冲突
    unique_id = int(time.time())
    temp_dir = os.path.dirname(output_file) if os.path.dirname(output_file) else '.'
    temp_part1 = os.path.join(temp_dir, f"temp_part1_{unique_id}.mp4")
    temp_part2 = os.path.join(temp_dir, f"temp_part2_{unique_id}.mp4")
    temp_list = os.path.join(temp_dir, f"temp_list_{unique_id}.txt")


    # --- 2. 查找关键帧 (I 帧) ---
    print("🔎 正在使用 FFprobe 查找关键帧...")
    try:
        # **修正后的 FFprobe 命令**：请求多个时间戳字段
        command = [
            'ffprobe',
            '-v', 'error', # 保持输出整洁
            '-select_streams', 'v:0',
            '-show_entries', 'frame=key_frame,pkt_pts_time,pts_time,best_effort_timestamp_time',
            '-of', 'json',
            input_file
        ]
        
        # 重新启用 check=True，确保命令正确执行
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        frames_data = json.loads(result.stdout)
        
        # **修正后的关键帧提取逻辑**：使用 get_frame_time 确保找到时间
        keyframes = sorted([
            time_sec
            for frame in frames_data.get('frames', []) 
            if frame.get('key_frame') == 1 and (time_sec := get_frame_time(frame)) is not None
        ])
        
        if not keyframes:
            print("❌ 错误：在视频流中找不到有效的关键帧时间戳。请检查文件是否为标准格式或是否损坏。")
            return
            
    except subprocess.CalledProcessError as e:
        print(f"❌ FFprobe 执行错误。请确认 FFprobe 已正确安装并可用。错误信息:\n{e.stderr}")
        return
    except json.JSONDecodeError:
        print("❌ FFprobe 输出格式不正确。")
        return
    
    # --- 3. 确定精确的无损切割时间点 ---
    
    # T_cut1: T_start_sec 之前最近的 I 帧（第一段的结束点）
    i = bisect.bisect_left(keyframes, T_start_sec)
    T_cut1_sec = keyframes[i - 1] if i > 0 else 0
    T_cut1_str = seconds_to_time(T_cut1_sec)
    
    # T_cut2: T_end_sec 之后最近的 I 帧（第二段的起始点）
    j = bisect.bisect_right(keyframes, T_end_sec)
    T_cut2_sec = keyframes[j] if j < len(keyframes) else keyframes[-1]
    T_cut2_str = seconds_to_time(T_cut2_sec)

    print(f"✅ 精确无损剪切点已确定：")
    print(f"  - 第一段结束 (原始:{t_start_str}) -> 实际切割点: {T_cut1_str}")
    print(f"  - 第二段开始 (原始:{t_end_str}) -> 实际切割点: {T_cut2_str}")
    
    # --- 4. 执行 FFmpeg 剪切和拼接 ---
    try:
        # A. 剪切第一段 (从开始到 T_cut1)
        print("\n✂️ 正在剪切第一段...")
        cmd1 = [
            'ffmpeg', '-i', input_file, 
            '-c', 'copy', '-to', T_cut1_str, 
            temp_part1
        ]
        subprocess.run(cmd1, capture_output=True, text=True, check=True)

        # B. 剪切第二段 (从 T_cut2 到结尾)
        print("✂️ 正在剪切第二段...")
        cmd2 = [
            'ffmpeg', '-ss', T_cut2_str, '-i', input_file, 
            '-c', 'copy', '-avoid_negative_ts', 'make_zero', 
            temp_part2
        ]
        subprocess.run(cmd2, capture_output=True, text=True, check=True)

        # C. 创建拼接列表文件
        with open(temp_list, 'w') as f:
            f.write(f"file '{os.path.basename(temp_part1)}'\n") # 写入相对路径
            f.write(f"file '{os.path.basename(temp_part2)}'\n")
            
        # D. 执行拼接，在临时文件所在目录执行，避免路径问题
        print("🔗 正在拼接两段视频...")
        cmd_concat = [
            'ffmpeg', '-f', 'concat', '-safe', '0', '-i', temp_list, 
            '-c', 'copy', output_file
        ]
        # 注意：此处使用 cwd 确保 ffmpeg 找到 list.txt 及其中的相对路径文件
        subprocess.run(cmd_concat, capture_output=True, text=True, check=True, cwd=temp_dir) 
        
        print(f"\n🎉 成功！文件已无损保存在: {output_file}")

    except subprocess.CalledProcessError as e:
        print(f"❌ FFmpeg 执行错误！请检查文件路径和权限。错误信息:\n{e.stderr}")
    
    finally:
        # --- 5. 清理临时文件 ---
        print("🧹 正在清理临时文件...")
        for f in [temp_part1, temp_part2, temp_list]:
            if os.path.exists(f):
                os.remove(f)

# --- 如何使用 ---
if __name__ == '__main__':
    # 替换为您的文件路径和时间
    INPUT_FILE = '/Volumes/sw/tts_result/tianchaoyaoyuan2/result_part3.mp4'  # <<< 替换为您的输入文件路径
    START_TIME = '01:34:28'                                                  # <<< 要删除区间的起始时间
    END_TIME = '01:35:52'                                                    # <<< 要删除区间的结束时间
    OUTPUT_FILE = '/Volumes/sw/tts_result/tianchaoyaoyuan2/result_clipped.mp4' # <<< 替换为您的输出文件路径

    if not os.path.exists(INPUT_FILE):
        print(f"错误: 输入文件未找到: {INPUT_FILE}")
    else:
        # 确保 FFmpeg/FFprobe 可用
        try:
            subprocess.run(['ffmpeg', '-version'], check=True, capture_output=True)
            subprocess.run(['ffprobe', '-version'], check=True, capture_output=True)
            cut_segment_lossless(INPUT_FILE, START_TIME, END_TIME, OUTPUT_FILE)
        except subprocess.CalledProcessError:
            print("请检查 FFmpeg 和 FFprobe 是否已正确安装并添加到系统 PATH 中。")
        except FileNotFoundError:
            print("找不到 FFmpeg/FFprobe 命令。请确保它们已安装。")

--- 任务开始: 裁剪区间 01:34:28 至 01:35:52 ---
🔎 正在使用 FFprobe 查找关键帧...
✅ 精确无损剪切点已确定：
  - 第一段结束 (原始:01:34:28) -> 实际切割点: 1:34:22.972005
  - 第二段开始 (原始:01:35:52) -> 实际切割点: 1:36:02.347005

✂️ 正在剪切第一段...
✂️ 正在剪切第二段...
🔗 正在拼接两段视频...

🎉 成功！文件已无损保存在: /Volumes/sw/tts_result/tianchaoyaoyuan2/result_clipped.mp4
🧹 正在清理临时文件...
